<a href="https://colab.research.google.com/github/tigerywh/colab/blob/master/langchain_ddup_share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

大语言模型集成工具 LangChain 

快速了解如何使用 LangChain 构建端到端语言模型应用程序。

参考官方文档： https://github.com/hwchase17/langchain/blob/master/docs/getting_started/getting_started.md

安装依赖 LangChain

In [ ]:
!pip install langchain
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


设置环境变量

In [ ]:

import os
os.environ["OPENAI_API_KEY"] = "..."

构建语言模型应用程序：LLM

In [ ]:
#In order to do this, we first need to import the LLM wrapper.

from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



Brightly Socks.


Chains：在多步骤工作流程中结合 LLM 和提示

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
chain.run("colorful socks")

'\n\nBrightSteps Socks.'

代理：基于用户输入的动态调用链
如果使用得当，代理可以非常强大。在本教程中，我们将向您展示如何通过最简单、最高级别的 API 轻松使用代理。

为了加载代理，您应该了解以下概念：

工具：执行特定任务的功能。这可以是：Google 搜索、数据库查找、Python REPL、其他链。工具的接口目前是一个函数，期望将字符串作为输入，将字符串作为输出。
LLM：为代理提供支持的语言模型。
代理：要使用的代理。这应该是一个引用支持代理类的字符串。由于本笔记本侧重于最简单、最高级别的 API，因此仅涵盖使用标准支持的代理。如果您想实施自定义代理，请参阅自定义代理的文档（即将推出）。
代理：有关受支持代理及其规范的列表，请参见此处。

工具：有关预定义工具及其规格的列表，请参见此处。

对于此示例，您还需要安装 SerpAPI Python 包。

In [ ]:
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32019 sha256=58f06553d3ce2b7c3a02a96f198e7280053cc04a9c54672550ac5f9e68347226
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [ ]:
import os
#访问 https://serpapi.com/，并单击页面右上角的“register”按钮注册获取SERPAPI_API_KEY，每月免费调用100次。
os.environ["SERPAPI_API_KEY"] = "..."


In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")



> Entering new AgentExecutor chain...
 I need to find the temperature first, then use the calculator to raise it to the .023 power.
Action: Search
Action Input: "High temperature in SF yesterday"
Observation: High: 53.6ºf @3:50 AM Low: 51.08ºf @12:56 AM Approx. Precipitation / Rain Total: 0.072 in. 1hr.
Thought: I now need to use the calculator to raise 53.6 to the .023 power
Action: Calculator
Action Input: 53.6^.023
Observation: Answer: 1.0958996548776438
Thought: I now know the final answer
Final Answer: The high temperature in SF yesterday in Fahrenheit raised to the .023 power is 1.0958996548776438.

> Finished chain.


'The high temperature in SF yesterday in Fahrenheit raised to the .023 power is 1.0958996548776438.'

In [ ]:
agent.run("北京昨天的最高温度多少? 这个温度的平方是多？")



> Entering new AgentExecutor chain...
 I need to find the temperature and then calculate its square.
Action: Search
Action Input: "Beijing yesterday temperature"
Observation: Beijing Temperature Yesterday. Maximum temperature yesterday: 75 °F (at 2:00 pm) Minimum temperature yesterday: 51 °F (at 5:00 am) Average temperature ...
Thought: I now need to calculate the square of the temperature.
Action: Calculator
Action Input: 75^2
Observation: Answer: 5625
Thought: I now know the final answer.
Final Answer: The maximum temperature in Beijing yesterday was 75°F and its square is 5625.

> Finished chain.


'The maximum temperature in Beijing yesterday was 75°F and its square is 5625.'

构建语言模型应用程序：聊天模型
同样，您可以使用聊天模型而不是 LLM。聊天模型是语言模型的变体。虽然聊天模型在底层使用语言模型，但它们公开的接口有点不同：它们公开的不是“文本输入、文本输出”API，而是“聊天消息”作为输入和输出的接口。

In [ ]:
#从聊天模型中获取消息完成
#您可以通过将一条或多条消息传递给聊天模型来获得聊天完成。响应将是一条消息。LangChain 
#目前支持的消息类型有AIMessage, HumanMessage, SystemMessage, 和ChatMessage--ChatMessage接受任意角色参数。大多数时候，您只会处理HumanMessage、AIMessage和SystemMessage。

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)

In [ ]:
chat([HumanMessage(content="Translate this sentence from English to French. I love programming.")])

AIMessage(content="J'aime programmer.", additional_kwargs={}, example=False)

In [ ]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming.")
]
chat(messages)

AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False)

In [ ]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(text="J'adore la programmation.", generation_info=None, message=AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False))], [ChatGeneration(text="J'adore l'intelligence artificielle.", generation_info=None, message=AIMessage(content="J'adore l'intelligence artificielle.", additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 57, 'completion_tokens': 20, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo'})

In [ ]:
result.llm_output['token_usage']

{'prompt_tokens': 57, 'completion_tokens': 20, 'total_tokens': 77}

In [ ]:
#聊天提示模板
#与 LLM 类似，您可以通过使用MessagePromptTemplate. 您可以ChatPromptTemplate从一个或多个MessagePromptTemplates 构建一个。
#您可以使用ChatPromptTemplate's format_prompt-- 这会返回一个PromptValue，您可以将其转换为字符串或Message对象，具体取决于您是否要将格式化值用作 llm 或聊天模型的输入。


from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language="English", output_language="Chinese", text="I love programming.").to_messages())
# -> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(content='我喜欢编程。', additional_kwargs={}, example=False)

In [ ]:
#带有聊天模型的链，LLMChain也可以与聊天模型一起使用：

from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="Chinese", text="I love programming.")

'我喜欢编程。'

In [ ]:
chain.run(input_language="English", output_language="Chinese", text=" Fahrenheit")

'华氏度'

In [ ]:
#代理也可以与聊天模型一起使用，您可以初始化一个AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION作为代理类型。

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("Who is Olivia Wilde  boyfriend? What is his current age raised to the 0.23 power?")



> Entering new AgentExecutor chain...
Thought: The first question requires a search, while the second question requires a calculator.

Action:
```
{
  "action": "Search",
  "action_input": "Olivia Wilde boyfriend"
}
```


Observation: Sudeikis and Wilde's relationship ended in November 2020. Wilde was publicly served with court documents regarding child custody while she was presenting Don't Worry Darling at CinemaCon 2022. In January 2021, Wilde began dating singer Harry Styles after meeting during the filming of Don't Worry Darling.
Thought:For the second question, I need to use the calculator tool.

Action:
```
{
  "action": "Calculator",
  "action_input": "(38 ** 0.23)"
}
```


Observation: Answer: 2.3086081644669734
Thought:I now know the final answer.

Final Answer: 2.3086081644669734

> Finished chain.


'2.3086081644669734'

In [ ]:
#内存：将状态添加到链和代理
#您可以将内存与使用聊天模型初始化的链和代理一起使用。这与 Memory for LLM 之间的主要区别在于，我们可以将它们保留为自己唯一的内存对象，而不是试图将所有以前的消息压缩成一个字符串。

from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

conversation.predict(input="Hi there!")
# -> 'Hello! How can I assist you today?'


conversation.predict(input="I'm doing well! Just having a conversation with an AI.")
# -> "That sounds like fun! I'm happy to chat with you. Is there anything specific you'd like to talk about?"

conversation.predict(input="Tell me about yourself.")
# -> "Sure! I am an AI language model created by OpenAI. I was trained on a large dataset of text from the internet, which allows me to understand and generate human-like language. I can answer questions, provide information, and even have conversations like this one. Is there anything else you'd like to know about me?"

"Sure! I am an AI language model created by OpenAI. I was trained on a large dataset of text from the internet, which allows me to understand and generate human-like language. I can answer questions, provide information, and even have conversations with people like you! Is there anything specific you'd like to know about me?"